# <span style="color:lightblue; font-weight:bold;">EXTRACCIÓ D'ENTITATS ANOMENADES</span>

## <span style="color:#ADD8E6; font-weight:bold;">Índex:</span>
### <span style="color:#FFFFFF;">1. [Inicialització](#init)</span>
### <span style="color:#FFFFFF;">2. [Predicció amb BIO](#bio)</span>
### <span style="color:#FFFFFF;">3. [Predicció amb IO](#io)</span>
### <span style="color:#ADD8E6;">4. [Predicció amb BIOES](#bioes)</span>
### <span style="color:#ADD8E6;">5. [Conclusions](#conclusions)</span>
### <span style="color:#ADD8E6;">6. [Execució dels models amb textos reals](#text_real)</span>
### <span style="color:#ADD8E6;">7. [Opcional CADEC](#opcional)</span>

## <span style="color:lightblue;"> Imports i Descarregues </span>

In [14]:
import nltk
import pycrfsuite
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from sklearn.metrics import accuracy_score
import unicodedata
import re

In [3]:
nltk.download('punkt') # Tokenitzador
nltk.download('averaged_perceptron_tagger') # Etiquetador POS
nltk.download('maxent_ne_chunker') # Etiquetador Entitats Anomenades
nltk.download('words')
nltk.download('treebank')
nltk.download('conll2002')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-d

True

## <span style="color:lightblue; font-weight:bold;">Inicialització</span> <a id="init"></a>

<div style="text-align: justify; max-width: 1250px">
En aquesta secció obtenim els conjunts que utilitzarem més endavant per entrenar i avaluar els nostres models de reconeixement d'entitats anomenades. 
A més, definim les funcions que utilitzarem a posteriori per automatitzar procesos repetitius.
</div>

In [4]:
train_esp = conll2002.iob_sents('esp.train') # Train
testa_esp = conll2002.iob_sents('esp.testa') # Dev
testb_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
testa_ned = conll2002.iob_sents('ned.testa') # Dev
testb_ned = conll2002.iob_sents('ned.testb') # Test

<div style="text-align: justify">
Definim les funcions per a obtenir diferents configuracions de representació dels nostres conjunts a entrenar i avaluar.
</div>

In [5]:
def obtenir_token_POS(fitxer):
    """
    Funció per convertir un text amb el token, POS tag i entitat per 
    cada element en cada frase en un text amb el token i el seu POS tag
    per cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem2))
        res.append(frases)
    return res

# Fem prediccions i mirem l'accuracy
def obtenir_token(fitxer):
    """
    Funció per convertir un text amb el token, POS tag i entitat 
    per cada element en cada frase en un text amb només el token
    per cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append(elem1)
        res.append(frases)
    return res


def obtenir_token_entity(fitxer):
    """
    Funció per convertir un text amb el token, POS tag i entitat per cada 
    element en cada frase en un text amb el token i la seva entitat per 
    cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2, elem3 in sentence:
            frases.append((elem1, elem3))
        res.append(frases)
    return res

### <span style="color:lightblue;"> Classe FeatureExtractor personalitzada </span>

<span> Features que es tenen en compte:
<ul>
    <li>Paraula actual</li>
    <li>Si comença en majúscula</li>
    <li>Si té signe de puntuació</li>
    <li>Si té números</li>
    <li>Prefixos fins a longitud 3</li>
    <li>Sufixos fins a longitud 3</li>
    <li>Paraules prèvies i posteriors amb POS-tags</li>
    <li>POS-tags</li>
</ul>
</span>

In [6]:
class FeatureExtractor:
    
    """
    Aquesta classe conté el mètode per calcular les features
    que s'utilitzaran per entrenar el model més endavant.
    """
    
    def __init__(self, use_basic_features=False, use_prefix_suffix_features=False, use_context_features=False, pattern = r'\d+', model_POS = None):
        self.use_basic_features = use_basic_features
        self.use_prefix_suffix_features = use_prefix_suffix_features
        self.use_context_features = use_context_features
        self._pattern = pattern
        self.model_POS = model_POS
        
    def __str__(self):
        features = []
        if self.use_basic_features:
            features.append("use_basic_features=True")
        if self.use_prefix_suffix_features:
            features.append("use_prefix_suffix_features=True")
        if self.use_context_features:
            features.append("use_context_features=True")
        return f"FeatureExtractor({', '.join(features)})"
        
    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
            
        token = tokens[idx]
        
        feature_list = []
        
        if self.use_basic_features:
            # Capitalization
            if token[0].isupper():
                feature_list.append("CAPITALIZATION")

            # Number
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM")

            # Punctuation
            punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION")

                    
        if self.use_prefix_suffix_features:
            # preffix up to length 3
            if len(token) > 1:
                feature_list.append("PRE_" + token[:1])
            if len(token) > 2:
                feature_list.append("PRE_" + token[:2])
            if len(token) > 3:
                feature_list.append("PRE_" + token[:3])

            # Suffix up to length 3
            if len(token) > 1:
                feature_list.append("SUF_" + token[-1:])
            if len(token) > 2:
                feature_list.append("SUF_" + token[-2:])
            if len(token) > 3:
                feature_list.append("SUF_" + token[-3:])
    
        
        if self.use_context_features:
            # POS_tags
            if self.model_POS:
                POS = self.model_POS.tag(tokens)
                
            # Paraules prèvies amb POS
            if idx > 0:
                feature_list.append("anterior1_" + tokens[idx-1] + "_" + POS[idx-1][1])
            if idx > 1:
                feature_list.append("anterior2_" + tokens[idx-2] + "_" + POS[idx-2][1])
                
            # Paraules posteriors amb POS
            if idx < (len(tokens)-1):
                feature_list.append("posterior1_" + tokens[idx+1] + "_" + POS[idx+1][1])
            if idx < (len(tokens)-2):
                feature_list.append("posterior2_" + tokens[idx+2] + "_" + POS[idx+2][1])

            feature_list.append("WORD_" + token + "_" + POS[idx][1])
            
        
        if not self.use_context_features:
            feature_list.append("WORD_" + token)
            
        
        return feature_list

### <span style="color:lightblue;"> Mètriques d'avaluació </span>

Aquestes funcions calculen la precisió, el recall i el F1-score per avaluar quan bé un model identifica les entitats en comparació amb les dades reals.

In [17]:
def calcular_precisio(entitats_referencia, entitats_predites):
    # Calcula el número d'entitats correctament identificades pel model
    entitats_correctes = entitats_referencia.intersection(entitats_predites)
    
    # Calcular la precisió
    if len(entitats_predites) > 0:
        precisio = len(entitats_correctes) / len(entitats_predites)
    else:
        precisio = 0.0
    
    return precisio


def calcular_recall(entitats_referencia, entitats_predites):
    # Calcula el número d'entitats correctament identificades pel model
    entitats_correctes = entitats_referencia.intersection(entitats_predites)
    
    # Calcular recall
    if len(entitats_referencia) > 0:
        recall = len(entitats_correctes) / len(entitats_referencia)
    else:
        recall = 0.0
    
    return recall

def calcular_f1_score(precisio, recall):
    # Calcular el F1-score
    if (precisio + recall) > 0:
        f1_score = 2 * (precisio * recall) / (precisio + recall)
    else:
        f1_score = 0.0
    
    return f1_score

def resultats(predicted_BIO, testa_esp_BIO_tag, obtain_entity):

    # Obtenir els conjunts d'entitats de referència i predites
    entitats_referencia = obtain_entity(testa_esp_BIO_tag)  # Conjunt d'entitats etiquetades manualment com a referència
    entitats_predites =  obtain_entity(predicted_BIO) # Conjunt d'entitats predites

    # Calcular la precisió
    precisio = calcular_precisio(entitats_referencia, entitats_predites)

    # Calcular la recall
    recall = calcular_recall(entitats_referencia, entitats_predites)

    # Calcular el F1-score
    f1_score = calcular_f1_score(precisio, recall)

    print("Precisió:", precisio)
    print("Recall:", recall)
    print("F1-score:", f1_score)


## <span style="color:lightblue; font-weight:bold;">Predicció amb BIO</span> <a id="bio"></a>

In [19]:
def obtenir_entitats_amb_posicions_BIO(fitxer_BIO_tag):
    """
    Funció per agrupar en sets les entitats anomenades, l'índex on comença i l'índex on acaba,
    i la classe de la entitat.
    
    Argument: un text amb una tupla (amb el token i la seva entitat) per cada element en cada frase.
    """
    
    entitats_amb_posicions = set()

    for sentence in fitxer_BIO_tag:
        ent = []
        name = None
        start_pos = None  # Posició d'inici de l'entitat actual
        prev_tag = None  # Guardar l'etiqueta del token anterior

        for token_index, token in enumerate(sentence):
            word, tag = token
            #word = word[0]

            if tag.startswith('B-'):
                # Si hi ha una entitat anterior, l'agreguem a la llista d'entitats
                if ent:
                    end_pos = token_index - 1  # La posició de fi es el token anterior
                    entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Creem una nova entitat amb la paraula actual
                ent = [word]
                # Obtenim el tipus d'entitat
                name = tag.split('-')[1]
                start_pos = token_index  # La posició d'inici es el token actual
                prev_tag = tag  # Actualitzem l'etiqueta del token anterior
            elif tag.startswith('I-'):
                # Només agreguem la paraula actual si el token anterior té etiqueta I- o B-
                if prev_tag:
                    ent.append(word)
                    prev_tag = tag  # Actualitzem l'etiqueta del token anterior
            elif tag == 'O' and ent:
                # Si trobem una etiqueta 'O' i hi ha una entitat en curs, l'agreguem a la llista d'entitats
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciem la llista de l'entitat actual
                ent = []
                prev_tag = None  # Reiniciem l'etiqueta del token anterior

        # Agreguem la última entitat si hi ha
        if ent:
            end_pos = len(sentence) - 1  # La posició de fi es l'últim token de la oració
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos)))

    return entitats_amb_posicions

#### ESP

<div style="text-align: justify; max-width: 1250px">
Comencem entrenant un model CRFTagger per a la predicció de les etiquetes POS, ja que serà utilitzat posteriorment per a calcular els features necessaris en el procés d'extracció d'entitats anomenades. Inicialment, vam considerar utilitzar les etiquetes POS existents en els textos com a dades d'entrada, però vam enfrontar-nos a un error durant el procés d'entrenament del model CRFTagger. Aquest error indicava que el model esperava rebre només un element amb la seva etiqueta, en comptes de dos elements amb una etiqueta, ja que estàvem passant una tupla que contenia tant el token com el seu POS, juntament amb l'etiqueta de l'entitat. Per tant, per evitar aquest problema, vam optar per entrenar un model separat per a la predicció de les etiquetes POS, que posteriorment seran utilitzades com a característiques addicionals en el procés d'entrenament per a l'extracció d'entitats anomenades.
</div>

Veiem que l'accuracy del model és prou elevada per tenir-lo en compte i utilitzar-lo més endavant.

In [13]:
model_tagger_POS_esp = CRFTagger()

# Entrenem el model per predir els POS que corresponen a cada token

train_esp_pos_tag = obtenir_token_POS(train_esp)
    
model_tagger_POS_esp.train(train_esp_pos_tag, 'model_POS_esp.crf.tagger')    


testa_esp_pre_tag = obtenir_token(testa_esp)
    
predicted = model_tagger_POS_esp.tag_sents(testa_esp_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_esp for elem in sentence]

print(accuracy_score(predictions, real_label))

0.9447121289420479


<div style="text-align: justify; max-width: 1250px">
A continuació, realitzem experiments amb diverses features per determinar quines proporcionen un rendiment òptim pel model amb etiquetatge BIO, i en textos escrits en espanyol. A més, aquests experiments els executem en el conjunt de proves "testa", que considerem com a conjunt de validació per ajustar i identificar les millors característiques.
</div>

In [14]:
param_combinations_esp = [
    None,
    FeatureExtractor(),
    FeatureExtractor(True, model_POS=model_tagger_POS_esp),
    FeatureExtractor(True, True, model_POS=model_tagger_POS_esp),
    FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)
]

train_esp_BIO = obtenir_token_entity(train_esp)

testa_esp_real = obtenir_token_entity(testa_esp)

def model_entrenament(train_esp_BIO_tag, extractor, testa_esp_pre_tag):
    if extractor == None:
        model_BIO = CRFTagger()
        model_BIO.train(train_esp_BIO_tag, 'model_BIO.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)
    
    else:
        model_BIO = CRFTagger(feature_func=extractor._get_features)
        model_BIO.train(train_esp_BIO_tag, 'model_BIO.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_esp_pre_tag)
    
    return resultats(predicted_BIO, testa_esp_real, obtenir_entitats_amb_posicions_BIO)

for param in param_combinations_esp:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_BIO, param, testa_esp_pre_tag)
    print('-'*50)
    

Model: Features per defecte de CRFTagger
Precisió: 0.6927890345649583
Recall: 0.643687707641196
F1-score: 0.6673363949483353
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.718299164768413
Recall: 0.2619047619047619
F1-score: 0.3838506796510448
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6447638603696099
Recall: 0.6085271317829457
F1-score: 0.626121635094716
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.7015675835551612
Recall: 0.6566998892580288
F1-score: 0.6783926783926784
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7475160724722385
Recall: 0.7081949058693244
F1-score: 0.7273244242251918
--------------------------------------------------


Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents i, per tant, usarem aquest model per a poder identificar les entitats correctament en el conjunt de test que no hem utilitzat fins ara, el 'testb'.

In [9]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_real = obtenir_token_entity(testb_esp)
testb_esp_pre_tag = obtenir_token(testb_esp)

model_BIO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO_esp.train(train_esp_BIO, 'model_BIO.crf.tagger')
    
predicted_BIO = model_BIO_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_BIO, testb_esp_real, obtenir_entitats_amb_posicions_BIO)


Precisió: 0.7869019341703427
Recall: 0.7645895153313551
F1-score: 0.7755852842809364


Els resultats són fins i tot millors que els obtinguts en el testa.

#### NED

Entrenem el model de CRF per predir els POS tags però ara pel nederlandés.

In [10]:
model_tagger_POS_ned = CRFTagger()

# Entrenem el model per predir els POS que corresponen a cada token
train_ned_pos_tag = obtenir_token_POS(train_ned)
    
model_tagger_POS_ned.train(train_ned_pos_tag, 'model_POS_ned.crf.tagger')


# Fem prediccions i mirem l'accuracy
testa_ned_pre_tag = obtenir_token(testa_ned)
    
predicted = model_tagger_POS_ned.tag_sents(testa_ned_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in testa_ned for elem in sentence]

print(accuracy_score(predictions, real_label))

0.940191577997718


Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIO' i la llengua neerlandesa.

In [11]:
param_combinations_ned = [
    None,
    FeatureExtractor(),
    FeatureExtractor(True, model_POS=model_tagger_POS_ned),
    FeatureExtractor(True, True, model_POS=model_tagger_POS_ned),
    FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)
]

train_ned_BIO = obtenir_token_entity(train_ned)

testa_ned_real = obtenir_token_entity(testa_ned)

def model_entrenament(train_BIO_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIO = CRFTagger()
        model_BIO.train(train_BIO_tag, 'model_BIO_ned.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_pre_tag)
    
    else:    
        model_BIO = CRFTagger(feature_func=extractor._get_features)
        model_BIO.train(train_BIO_tag, 'model_BIO_ned.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(testa_pre_tag)
    
    return resultats(predicted_BIO, testa_ned_real, obtenir_entitats_amb_posicions_BIO)

for param in param_combinations_ned:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_BIO, param, testa_ned_pre_tag)
    print('-'*50)

Model: Features per defecte de CRFTagger
Precisió: 0.6441908713692946
Recall: 0.565059144676979
F1-score: 0.602035870092099
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.762114537444934
Recall: 0.15741583257506825
F1-score: 0.2609351432880845
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.655096011816839
Recall: 0.4035486806187443
F1-score: 0.49943693693693697
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6853182751540041
Recall: 0.6073703366696998
F1-score: 0.6439942112879885
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7082306554953179
Recall: 0.6537761601455869
F1-score: 0.6799148332150462
--------------------------------------------------


Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents i, per tant, usarem aquest model per a poder identificar les entitats correctament.

In [12]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_real = obtenir_token_entity(testb_ned)
testb_ned_pre_tag = obtenir_token(testb_ned)

model_BIO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIO_ned.train(train_ned_BIO, 'model_BIO_ned.crf.tagger')
    
predicted_BIO = model_BIO_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_BIO, testb_ned_real, obtenir_entitats_amb_posicions_BIO)

Precisió: 0.7402511566424322
Recall: 0.6860643185298622
F1-score: 0.7121284374503258


Els resultats tornen a ser novament millors que els obtinguts en el testa.

## <span style="color:lightblue; font-weight:bold;">Predicció amb IO</span> <a id="io"></a>

Definim una funció per passar de l'etiquetatge per defecte 'BIO' a l'etiquetatge 'IO'.

In [13]:
def convert_to_io(train_data_bio):
    """
    Funció per convertir les dades del format BIO al format IO.
    Argument:
    train_data_bio: una llista de frases, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIO).
    Retorna:
    Una llista de frases en format IO, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta IO).
    """
    train_data_io = []
    for sentence in train_data_bio:
        io_tags = []
        for word, pos_tag, bio_tag in sentence:
            if bio_tag == 'O':
                io_tags.append('O')
            elif bio_tag.startswith('B-'):
                io_tags.append('I' + bio_tag[1:])
            else:
                io_tags.append(bio_tag)
        
        train_data_io.append([(word, pos_tag, io_tag) for (word, pos_tag, bio_tag), io_tag in zip(sentence, io_tags)])
    return train_data_io

<div style="text-align: justify; max-width: 1250px;">
Aquesta funció agrupa les entitats anomenades amb les seves posicions d'inici i fi, així com la seva classe, en un conjunt de tuples. Cada tuple conté tres elements: la llista de tokens que formen l'entitat, una tupla amb les posicions d'inici i fi (els índexs) de l'entitat dins del text, i la classe de l'entitat.
</div>

In [24]:
def obtenir_entitats_amb_posicions_IO(fitxer_IO_tag):
    """
    Funció per a agrupar en sets les entitats anomenades, l'índex on comença i l'índex on acaba,
    i la classe de l'entitat.
    
    Argument: un text amb una tupla (amb el token i la seva entitat) per a cada element en cada frase.
    """
    
    entitats_amb_posicions = set()

    for sentence_index, sentence in enumerate(fitxer_IO_tag):
        ent = []
        name = None
        start_pos = None  # Posició de inici de la entitat actual
        prev_tag = None  # Guardar l'etiqueta del token anterior

        for token_index, token in enumerate(sentence):
            word, tag = token

            if tag.startswith('I-'):
                # Si l'etiqueta es 'I-' i el token anterior es 'O', comencem una nova entitat
                if not prev_tag:
                    # Si hi ha una entitat anterior, l'afegim a la llista d'entitats
                    if ent:
                        end_pos = token_index - 1  # La posició de fi es el token anterior
                        entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                    # Creem una nova entitat amb la palabra actual
                    ent = [word]
                    # Obtenim el tipus de entitat
                    name = tag.split('-')[1]
                    start_pos = token_index  # La posició de inici es el token actual
                else:
                    # Si el token anterior es 'I-', agreguem la paraula actual a la entitat en curs
                    ent.append(word)
                prev_tag = tag  # Actualitzem l'etiqueta del token anterior
            
            elif tag == 'O' and ent:
                # Si trobem una etiqueta 'O' i hi ha una entitat en curs, l'agreguem a la llista d'entitats
                end_pos = token_index - 1  # La posición de fin es el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciem la llista de l'entitat actual
                ent = []
                prev_tag = None  # Reiniciem l'etiqueta del token anterior

        # Agreguem la última entitat si hi ha
        if ent:
            end_pos = len(sentence) - 1  # La posició de fi es l'últim token de la oració
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))

    return entitats_amb_posicions


#### ESP

<div style="text-align: justify; max-width: 1250px;">
Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'IO' i la llengua espanyola, el mateix que hem fet amb la codificació 'BIO'.
</div>

In [15]:
train_esp_io = convert_to_io(train_esp)
train_esp_IO = obtenir_token_entity(train_esp_io)

testa_esp_io = convert_to_io(testa_esp)
testa_esp_real = obtenir_token_entity(testa_esp_io)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_IO = CRFTagger()
        model_IO.train(train_tag, 'model_IO.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    else:
        model_IO = CRFTagger(feature_func=extractor._get_features)
        model_IO.train(train_tag, 'model_IO.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    return resultats(predicted_IO, testa_esp_real, obtenir_entitats_amb_posicions_IO)

for param in param_combinations_esp: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_IO, param, testa_esp_pre_tag)
    print('-'*50)

Model: Features per defecte de CRFTagger
Precisió: 0.6720413751140858
Recall: 0.6208544125913434
F1-score: 0.6454346238130021
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.6519083969465649
Recall: 0.24002248454187747
F1-score: 0.35086277732128185
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6396209653538644
Recall: 0.6070826306913997
F1-score: 0.622927180966114
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6832579185520362
Recall: 0.636593591905565
F1-score: 0.6591008293321694
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.738569753810082
Recall: 0.7082630691399663
F1-score: 0.7230989956958392
--------------------------------------------------


<div style="text-align: justify; max-width: 1250px;">
Observem que els millors resultats es donen amb el model que usa els 3 'features' diferents, com en el cas anterior. Farem servir el model que utilitza aquestes 3 'features' per a poder identificar les entitats correctament en el 'testb'.
</div>

In [16]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_io = convert_to_io(testb_esp)
testb_esp_real = obtenir_token_entity(testb_esp_io)

testb_esp_pre_tag = obtenir_token(testb_esp)

model_IO_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_esp.train(train_esp_IO, 'model_IO.crf.tagger')
    
predicted_IO = model_IO_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_IO, testb_esp_real, obtenir_entitats_amb_posicions_IO)


Precisió: 0.779678412589805
Recall: 0.7553861451773285
F1-score: 0.7673400673400673


Novament els resultats són millors que els calculats sobre el 'testa'.

#### NED

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'IO' i la llengua neerlandesa.

In [17]:
train_ned_io = convert_to_io(train_ned)
train_ned_IO = obtenir_token_entity(train_ned_io)

testa_ned_io = convert_to_io(testa_ned)
testa_ned_real = obtenir_token_entity(testa_ned_io)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_IO = CRFTagger()
        model_IO.train(train_tag, 'model_io_ned.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    else:
        model_IO = CRFTagger(feature_func=extractor._get_features)
        model_IO.train(train_tag, 'model_io_ned.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(testa_pre_tag)
        
    return resultats(predicted_IO, testa_ned_real, obtenir_entitats_amb_posicions_IO)

for param in param_combinations_ned: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_IO, param, testa_ned_pre_tag)
    print('-'*50)

Model: Features per defecte de CRFTagger
Precisió: 0.6347177848775293
Recall: 0.5614696184644371
F1-score: 0.5958510372406899
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.7369727047146402
Recall: 0.13989637305699482
F1-score: 0.2351543942992874
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.623082542001461
Recall: 0.401789919924635
F1-score: 0.4885452462772051
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6628211851074987
Recall: 0.5953838907206783
F1-score: 0.6272952853598015
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7160931174089069
Recall: 0.6665096561469619
F1-score: 0.6904122956818737
--------------------------------------------------


<div style="text-align: justify; max-width: 1250px;">
En aquesta ocasió tornem a observar que els millors resultats es donen amb el model que usa els 3 'features' diferents, per tant, farem servir el model que utilitza aquestes 3 'features' per a poder identificar les entitats correctament en el 'testb'.
</div>

In [18]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_io = convert_to_io(testb_ned)
testb_ned_real = obtenir_token_entity(testb_ned_io)

testb_ned_pre_tag = obtenir_token(testb_ned)

model_IO_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_IO_ned.train(train_ned_IO, 'model_io_ned.crf.tagger')
    
predicted_IO = model_IO_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_IO, testb_ned_real, obtenir_entitats_amb_posicions_IO)

Precisió: 0.7119216480918609
Recall: 0.670057215511761
F1-score: 0.6903553299492386


## <span style="color:lightblue; font-weight:bold;">Predicció amb BIOES</span> <a id="bioes"></a>

In [19]:
def convert_to_bioes(train_data_bio):
    """
    Funció per convertir dades de format BIO a format BIOES.
    
    Argument:
    train_data_bio: una llista de frases, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIO).
    
    Retorna:
    Una llista de frases en format BIOES, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIOES).
    """
    train_data_bioes = []
    for sentence in train_data_bio:
        bioes_tags = []
        for i, (word, pos_tag, bio_tag) in enumerate(sentence):
            if bio_tag == 'O':
                bioes_tags.append('O')
            elif bio_tag.startswith('B-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('S' + bio_tag[1:])  # Single
                else:
                    bioes_tags.append('B' + bio_tag[1:])  # Begin
            elif bio_tag.startswith('I-'):
                if i == len(sentence) - 1 or sentence[i + 1][2] != 'I' + bio_tag[1:]:
                    bioes_tags.append('E' + bio_tag[1:])  # End
                else:
                    bioes_tags.append('I' + bio_tag[1:])  # Inside
            else:
                raise ValueError("Etiqueta BIO incorrecta: {}".format(bio_tag))
        
        train_data_bioes.append([(word, pos_tag, bioes_tag) for (word, pos_tag, bio_tag), bioes_tag in zip(sentence, bioes_tags)])
    return train_data_bioes

In [28]:
def obtenir_entitats_amb_posicions_bioes(train_data_bioes):
    """
    Funció per agrupar a sets les entitats nomenades, l'índex on comença i l'índex on acaba,
    i la classe de lentitat.

    Argument: un text amb una tupla (amb el token i la seva entitat) per a cada element a cada frase.
    """
        
    entitats_amb_posicions = set()

    for sentence_index, sentence in enumerate(train_data_bioes):
        ent = []
        name = None
        start_pos = None  # Posició d'inici de l'entitat actual

        for token_index, (word, bioes_tag) in enumerate(sentence):
            if bioes_tag != 'O':
                if bioes_tag.startswith('B-') or bioes_tag.startswith('S-'):
                    # Si hi ha una entitat anterior, l'afegim a la llista d'entitats
                    if ent:
                        end_pos = token_index - 1  # La posició de fi és el token anterior
                        entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                    # Creem una nova entitat amb la paraula actual
                    ent = [word]
                    # Obtenim el tipus d'entitat
                    name = bioes_tag.split('-')[1]
                    start_pos = token_index  # La posició inicial és el token actual
                elif bioes_tag.startswith('I-') or bioes_tag.startswith('E-'):
                    ent.append(word)
            elif ent:
                # Si trobem una etiqueta 'O' i hi ha una entitat en curs, l'afegim a la llista d'entitats
                end_pos = token_index - 1  # La posició de fi és el token anterior
                entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))
                # Reiniciem la llista de l'entitat actual
                ent = []

        # Agreguem la darrera entitat si n'hi ha
        if ent:
            end_pos = len(sentence) - 1  # La posició de fi és el darrer token de l'oració
            entitats_amb_posicions.add((tuple(ent), (start_pos, end_pos), name))

    return entitats_amb_posicions


#### ESP

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIOES' i la llengua espanyola.

In [21]:
train_esp_bioes = convert_to_bioes(train_esp)
train_esp_BIOES = obtenir_token_entity(train_esp_bioes)

testa_esp_bioes = convert_to_bioes(testa_esp)
testa_esp_real = obtenir_token_entity(testa_esp_bioes)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIOES = CRFTagger()
        model_BIOES.train(train_tag, 'model_BIOES_esp.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    else:
        model_BIOES = CRFTagger(feature_func=extractor._get_features)
        model_BIOES.train(train_tag, 'model_BIOES_esp.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    return resultats(predicted_BIOES, testa_esp_real, obtenir_entitats_amb_posicions_bioes)

for param in param_combinations_esp: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_esp_BIOES, param, testa_esp_pre_tag)
    print('-'*50)

Model: Features per defecte de CRFTagger
Precisió: 0.6887340301974448
Recall: 0.6565181289786881
F1-score: 0.6722403287515941
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.7877838684416602
Recall: 0.2784389703847218
F1-score: 0.4114519427402863
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6522241478913923
Recall: 0.6249654027124274
F1-score: 0.638303886925795
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.7036930178880554
Recall: 0.6750622751176307
F1-score: 0.6890803785845458
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7442737025224703
Recall: 0.7104898975920287
F1-score: 0.7269895213820448
--------------------------------------------------


<div style="text-align: justify; max-width: 1250px;">
Tornem a veure com el model que usa el 3 'features' obte els millors resultats i en aquesta ocasió tornarem a utilitzar el model que utilitza els 3 'features' per a identificar les entitats correctament.
</div>

In [22]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_esp)

testb_esp_bioes = convert_to_bioes(testb_esp)
testb_esp_real = obtenir_token_entity(testb_esp_bioes)

testb_esp_pre_tag = obtenir_token(testb_esp)

model_BIOES_esp = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_esp.train(train_esp_BIOES, 'model_BIOES_esp.crf.tagger')
    
predicted_BIOES = model_BIOES_esp.tag_sents(testb_esp_pre_tag)

resultats(predicted_BIOES, testb_esp_real, obtenir_entitats_amb_posicions_bioes)

Precisió: 0.7817327065144393
Recall: 0.7673038892551087
F1-score: 0.7744510978043911


Els resultats són gairebé els mateixos que els que hem obtingut amb la codificació 'BIO'.

#### NED

Experimentació amb diferents 'features' per a veure quins són els més adequats pel model amb 'BIOES' i la llengua neerlandesa.

In [23]:
train_ned_bioes = convert_to_bioes(train_ned)
train_ned_BIOES = obtenir_token_entity(train_ned_bioes)

testa_ned_bioes = convert_to_bioes(testa_ned)
testa_ned_real = obtenir_token_entity(testa_ned_bioes)

def model_entrenament(train_tag, extractor, testa_pre_tag):
    if extractor == None:
        model_BIOES = CRFTagger()
        model_BIOES.train(train_tag, 'model_BIOES_ned.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    else:
        model_BIOES = CRFTagger(feature_func=extractor._get_features)
        model_BIOES.train(train_tag, 'model_BIOES_ned.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(testa_pre_tag)
        
    return resultats(predicted_BIOES, testa_ned_real, obtenir_entitats_amb_posicions_bioes)

for param in param_combinations_ned: # definida primerament en la predicció del BIO
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_ned_BIOES, param, testa_ned_pre_tag)
    print('-'*50)

Model: Features per defecte de CRFTagger
Precisió: 0.6562178828365879
Recall: 0.5828388863532633
F1-score: 0.6173555716702925
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.7785087719298246
Recall: 0.16202647193062528
F1-score: 0.26822818284850775
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6494252873563219
Recall: 0.41259698767685987
F1-score: 0.504605079542283
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6781725888324873
Recall: 0.6097672295755363
F1-score: 0.6421533285267964
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, use_context_features=True)
Precisió: 0.7135802469135802
Recall: 0.659516202647193
F1-score: 0.6854838709677419
--------------------------------------------------


<div style="text-align: justify; max-width: 1250px;">
En aquest últim model veiem com una vegada més els millors resultats ens el dona el model que usa els 3 'features' i, per tant, una vegada més utilitzarem aquest model per identificar correctament les identitats en el 'testb'.
</div>

In [24]:
feature_extractor = FeatureExtractor(True, True, True, model_POS=model_tagger_POS_ned)

testb_ned_bioes = convert_to_bioes(testb_ned)
testb_ned_real = obtenir_token_entity(testb_ned_bioes)

testb_ned_pre_tag = obtenir_token(testb_ned)

model_BIOES_ned = CRFTagger(feature_func=feature_extractor._get_features)
model_BIOES_ned.train(train_ned_BIOES, 'model_BIOES_ned.crf.tagger')
    
predicted_BIOES = model_BIOES_ned.tag_sents(testb_ned_pre_tag)

resultats(predicted_BIOES, testb_ned_real, obtenir_entitats_amb_posicions_bioes)

Precisió: 0.7451505016722408
Recall: 0.6853275915103045
F1-score: 0.7139881429258131


Amb la codificació 'BIO' hem obtingut resultats molt semblants sobre aquest mateix 'testb'.

## <span style="color:lightblue; font-weight:bold;">CONCLUSIONS</span> <a id="conclusions"></a>

<div style="text-align: justify; max-width: 1250px;">
Després de les proves realitzades, es pot observar clarament que l'ús de tots els múltiples 'features' en un model millora significativament la identificació d'entitats. Aquesta millora és notable en comparació amb models que utilitzen menys característiques o cap, destacant la importància d'aquesta pràctica en el desenvolupament de models precisos entrenats amb molts atributs.

Pel que fa a les codificacions BIO, IO i BIOES en les tasques d'etiquetatge d'entitats anomenades, les proves mostren que proporcionen resultats molt similars, sense destacar notablement l'una sobre les altres. Tot i això, creiem que és important assenyalar que la codificació IO tendeix a obtenir lleugerament resultats pitjors en comparació amb les altres dues, tot i que aquesta diferència no és significativa o notòria. Podem atribuir aquest fet a la menor quantitat d'informació proporcionada per la codificació IO en la detecció d'entitats, limitant així les opcions dels models predictius en la seva capacitat per detectar-les o no.

Quant a la comparació entre els models en castellà i neerlandès, és evident que els models en castellà mostren millors resultats. Aquesta diferència pot atribuir-se a diversos factors, com ara la disponibilitat de dades etiquetades i corpus de text, la complexitat lingüística i els esforços de recerca i desenvolupament en cada idioma.

En resum, és important l'ús de múltiples 'features' en els models, les codificacions 'BIO', 'IO' i 'BIOES' donen resultats molt semblants. Tot i així, es pot observar un rendiment lleugerament inferior amb la codificació 'IO' en comparació amb les altres. A més, els models entrenats en castellà demostren un millor rendiment en comparació amb els models en neerlandès.
</div>

## <span style="color:lightblue; font-weight:bold;">Execució dels models amb textos reals</span> <a id="text_real"></a>

Per veure com funcionen els nostres models amb textos reals, hem utilitzat els següents dos textos en castellà i neerlandès respectivament.

<div style="text-align: justify; max-width: 1250px;">
'En la soleada ciudad de Barcelona, España, se celebró una emocionante competición de tenis. Rafael Nadal, el famoso tenista español, dominó la cancha con su habilidad característica. Los espectadores animaban con entusiasmo mientras Nadal avanzaba hacia la victoria. Al final del torneo, Nadal levantó el trofeo en medio de aplausos y ovaciones.'
</div>

<div style="text-align: justify; max-width: 1250px;">
'In de bruisende stad Amsterdam, Nederland, vond een spannende fietsrace plaats. Tom Dumoulin, de beroemde Nederlandse wielrenner, schitterde op de route met zijn krachtige pedaalslagen. Toeschouwers juichten hem toe terwijl Dumoulin naar de overwinning fietste. Aan het einde van de race stond Dumoulin trots op het podium, omgeven door applaus en felicitaties.'
</div>

In [25]:
# Definim ara funcions per tractar els textos reals per a que puguin ser predits pels nostres models.

def transformar(text):
    """
    Funció per transformar un text en una llista de frases.

    Argument:
    text: una cadena de caràcters que conté el text.

    Retorna:
    Una llista de frases, on cada frase és una llista de paraules del text original.
    """
    
    output = []
    frase = []
    palabras = text.split()
    
    for palabra in palabras:
        frase.append(palabra)
        if '.' in palabra:
            output.append(frase)
            frase = []
    
    if frase:
        output.append(frase)
    
    return output

def entitats(tupla):
    """
    Retorna l'entitat d'una tupla que conté les entitats en la primera posició
    """
    return tupla[0]


In [26]:
def etiquetar_paraules(text, paraules_etiqueta, codificacio):
    """
    Funció per etiquetar correctament les entitats anomenades d'un 
    text per cada token en una llista de frases.

    Argument:
    text: una llista de tokens per cada frase.
    paraules_etiqueta: una llista de tuples amb els tokens que formen les entitats i la seva classe.
    codificacio: el tipus de codificacio que volem obtenir.

    Retorna:
    Una llista de frases, on cada frase és una llista de tuples amb el token i la entitat anomenada.
    """
    
    etiquetes = []
    prev_tag = 'O'
    for sentence in text:
        etiquetes_oracio = []
        for i, token in enumerate(sentence):
            etiqueta = 'O'
            for paraula, etiqueta_paraula in paraules_etiqueta:
                if token == paraula:
                    etiqueta = etiqueta_paraula
                    break
            etiquetes_oracio.append([token, etiqueta])
            
            
        for i, etiqueta in enumerate(etiquetes_oracio):
            if codificacio == 'BIO':
                if etiqueta[1] != 'O':
                    if prev_tag == 'O':
                        etiqueta[1] = 'B-' + etiqueta[1]
                    else:
                        etiqueta[1] = 'I-' + etiqueta[1]
                prev_tag = etiqueta[1]
            
            elif codificacio == 'IO':
                if etiqueta[1] != 'O':
                    etiqueta[1] = 'I-' + etiqueta[1]
            
            elif codificacio == 'BIOES':
                if etiqueta[1] != 'O':
                    if i == 0 or prev_tag == 'O':
                        if i == len(sentence) - 1 or prev_tag == 'O' or sentence[i + 1][1] != etiqueta[1]:
                            etiqueta[1] = 'S-' + etiqueta[1]  # Single
                        else:
                            etiqueta[1] = 'B-' + etiqueta[1]  # Begin
                    else:
                        if i == len(sentence) - 1 or sentence[i + 1][1] == 'O' or sentence[i + 1][1] != etiqueta[1]:
                            etiqueta[1] = 'E-' + etiqueta[1]  # End
                        else:
                            etiqueta[1] = 'I-' + etiqueta[1]  # Inside
                prev_tag = etiqueta[1]
                
            etiquetes_oracio[i] = tuple(etiqueta)
            
        etiquetes.append(etiquetes_oracio)
    return etiquetes

In [27]:
# Guardem els textos en variables que indiquen l'idioma de cada text 

esp = 'En la soleada ciudad de Barcelona, España, se celebró una emocionante competición de tenis. Rafael Nadal, el famoso tenista español, dominó la cancha con su habilidad característica. Los espectadores animaban con entusiasmo mientras Nadal avanzaba hacia la victoria. Al final del torneo, Nadal levantó el trofeo en medio de aplausos y ovaciones.'
ned = 'In de bruisende stad Amsterdam, Nederland, vond een spannende fietsrace plaats. Tom Dumoulin, de beroemde Nederlandse wielrenner, schitterde op de route met zijn krachtige pedaalslagen. Toeschouwers juichten hem toe terwijl Dumoulin naar de overwinning fietste. Aan het einde van de race stond Dumoulin trots op het podium, omgeven door applaus en felicitaties.'

In [28]:
# Tokenitzem els textos amb la funció definida prèviament i creem els textos amb les etiquetes correctament definides

esp = transformar(esp)

palabras_etiqueta_esp = [('Barcelona,', 'LOC'), ('España,', 'LOC'), ('Rafael', 'PER'), ('Nadal,', 'PER'), ('Nadal', 'PER')]
text_real_BIO_esp = etiquetar_paraules(esp, palabras_etiqueta_esp, codificacio='BIO') # etiquetatge BIO
text_real_IO_esp = etiquetar_paraules(esp, palabras_etiqueta_esp, codificacio='IO') # etiquetatge IO
text_real_BIOES_esp = etiquetar_paraules(esp, palabras_etiqueta_esp, codificacio='BIOES') # etiquetatge BIOES



ned = transformar(ned)

palabras_etiqueta_ned = [('Amsterdam,', 'LOC'), ('Nederland,', 'LOC'), ('Tom', 'PER'), ('Dumoulin,', 'PER'), ('Nederlandse', 'MISC'), ('Dumoulin', 'PER')]
text_real_BIO_ned = etiquetar_paraules(ned, palabras_etiqueta_ned, codificacio='BIO') # etiquetatge BIO
text_real_IO_ned = etiquetar_paraules(ned, palabras_etiqueta_ned, codificacio='IO') # etiquetatge IO
text_real_BIOES_ned = etiquetar_paraules(ned, palabras_etiqueta_ned, codificacio='BIOES') # etiquetatge BIOES


### ESP

In [33]:
# Fem la predicció de l'etiquetat d'entitats amb els diferents models per les diferents codificacions

predicted_BIO_esp = model_BIO_esp.tag_sents(esp)
print(f'BIO: {predicted_BIO_esp}')

predicted_IO_esp = model_IO_esp.tag_sents(esp)
print(f'IO: {predicted_IO_esp}')

predicted_BIOES_esp = model_BIOES_esp.tag_sents(esp)
print(f'BIOES: {predicted_BIOES_esp}')

BIO: [[('En', 'O'), ('la', 'O'), ('soleada', 'O'), ('ciudad', 'O'), ('de', 'O'), ('Barcelona,', 'B-LOC'), ('España,', 'I-LOC'), ('se', 'O'), ('celebró', 'O'), ('una', 'O'), ('emocionante', 'O'), ('competición', 'O'), ('de', 'O'), ('tenis.', 'B-MISC')], [('Rafael', 'B-PER'), ('Nadal,', 'I-PER'), ('el', 'O'), ('famoso', 'O'), ('tenista', 'O'), ('español,', 'O'), ('dominó', 'O'), ('la', 'O'), ('cancha', 'O'), ('con', 'O'), ('su', 'O'), ('habilidad', 'O'), ('característica.', 'B-MISC')], [('Los', 'O'), ('espectadores', 'O'), ('animaban', 'O'), ('con', 'O'), ('entusiasmo', 'O'), ('mientras', 'O'), ('Nadal', 'O'), ('avanzaba', 'O'), ('hacia', 'O'), ('la', 'O'), ('victoria.', 'B-ORG')], [('Al', 'O'), ('final', 'O'), ('del', 'O'), ('torneo,', 'O'), ('Nadal', 'O'), ('levantó', 'O'), ('el', 'O'), ('trofeo', 'O'), ('en', 'O'), ('medio', 'O'), ('de', 'O'), ('aplausos', 'O'), ('y', 'O'), ('ovaciones.', 'B-MISC')]]
IO: [[('En', 'O'), ('la', 'O'), ('soleada', 'O'), ('ciudad', 'O'), ('de', 'O'), ('Bar

<div style="text-align: justify; max-width: 1250px;">
Observant les frases resultants amb els tokens etiquetats per tipus d'entitat per les tres diferents codificacions podem extreure que les dues primeres codificacions, 'BIO' i 'IO', tenen automatitzada la reconeixença de les últimes paraules, és a dir, les que acaben en punt, com a entitats anomenades. Malgrat això totes reconeixen correctament Rafael Nadal com una entitat de classe PER, però cap d'elles reconeix l'altre entitat 'Nadal' de la classe PER.
Finalment, la ciutat Barcelona i Espanya és reconeguda correctament com a entitat de classe LOC per les codificacions 'BIO' i 'IO', però, tot i que és reconeguda per la codificació 'BIOES' com a entitat, la classe predita d'aquesta entitat no és la correcte. 
</div>

Calculem ara els resultats de les prediccions.

In [30]:
esp_bio = obtenir_entitats_amb_posicions_BIO(predicted_BIO_esp)
palabras_y_etiqueta = [entitats(tupla) for tupla in esp_bio]
print('Etiquetatge BIO:', palabras_y_etiqueta)

esp_io = obtenir_entitats_amb_posicions_IO(predicted_IO_esp)
palabras_y_etiqueta = [entitats(tupla) for tupla in esp_io]
print('Etiquetatge IO:', palabras_y_etiqueta)

esp_bioes = obtenir_entitats_amb_posicions_bioes(predicted_BIOES_esp)
palabras_y_etiqueta = [entitats(tupla) for tupla in esp_bioes]
print('Etiquetatge BIOES:', palabras_y_etiqueta)

Etiquetatge BIO: [('característica.',), ('Barcelona,', 'España,'), ('victoria.',), ('Rafael', 'Nadal,'), ('tenis.',), ('ovaciones.',)]
Etiquetatge IO: [('Barcelona,', 'España,'), ('característica.',), ('Rafael', 'Nadal,'), ('ovaciones.',), ('tenis.',), ('victoria.',)]
Etiquetatge BIOES: [('ovaciones.',), ('Barcelona,', 'España,'), ('Rafael', 'Nadal,')]


In [31]:
print('Etiquetatge BIO:')
resultats(predicted_BIO_esp, text_real_BIO_esp, obtenir_entitats_amb_posicions_BIO)
print('-'*50)

print('\nEtiquetatge IO:')
resultats(predicted_IO_esp, text_real_IO_esp, obtenir_entitats_amb_posicions_IO)
print('-'*50)

print('\nEtiquetatge BIOES:')
resultats(predicted_BIOES_esp, text_real_BIOES_esp, obtenir_entitats_amb_posicions_bioes)
print('-'*50)

Etiquetatge BIO:
Precisió: 0.3333333333333333
Recall: 0.5
F1-score: 0.4
--------------------------------------------------

Etiquetatge IO:
Precisió: 0.3333333333333333
Recall: 0.5
F1-score: 0.4
--------------------------------------------------

Etiquetatge BIOES:
Precisió: 0.3333333333333333
Recall: 0.25
F1-score: 0.28571428571428575
--------------------------------------------------


<div style="text-align: justify; max-width: 1250px;">
Veiem que les codificacions que millor reconeixen entitats són les codificacions BIO i IO, tot i que no presenten bons resultats. Concloem que es deu a les poques dades utilitzades per fer les prediccions amb el text real, creiem que amb un text real més gran les prediccions augmentarien i on les codificacions BIO i BIOES tindrien millors resultats, tal com hem vist amb el còrpora de 'conll'.
</div>

### NED

In [32]:
predicted_BIO_ned = model_BIO_ned.tag_sents(ned)
print(f'BIO: {predicted_BIO_ned}')

predicted_IO_ned = model_IO_ned.tag_sents(ned)
print(f'IO: {predicted_IO_ned}')

predicted_BIOES_ned = model_BIOES_ned.tag_sents(ned)
print(f'BIOES: {predicted_BIOES_ned}')

BIO: [[('In', 'O'), ('de', 'O'), ('bruisende', 'O'), ('stad', 'O'), ('Amsterdam,', 'B-PER'), ('Nederland,', 'I-PER'), ('vond', 'O'), ('een', 'O'), ('spannende', 'O'), ('fietsrace', 'O'), ('plaats.', 'O')], [('Tom', 'B-PER'), ('Dumoulin,', 'I-PER'), ('de', 'O'), ('beroemde', 'O'), ('Nederlandse', 'B-MISC'), ('wielrenner,', 'O'), ('schitterde', 'O'), ('op', 'O'), ('de', 'O'), ('route', 'O'), ('met', 'O'), ('zijn', 'O'), ('krachtige', 'O'), ('pedaalslagen.', 'O')], [('Toeschouwers', 'O'), ('juichten', 'O'), ('hem', 'O'), ('toe', 'O'), ('terwijl', 'O'), ('Dumoulin', 'B-PER'), ('naar', 'O'), ('de', 'O'), ('overwinning', 'O'), ('fietste.', 'O')], [('Aan', 'O'), ('het', 'O'), ('einde', 'O'), ('van', 'O'), ('de', 'O'), ('race', 'O'), ('stond', 'O'), ('Dumoulin', 'B-PER'), ('trots', 'O'), ('op', 'O'), ('het', 'O'), ('podium,', 'O'), ('omgeven', 'O'), ('door', 'O'), ('applaus', 'O'), ('en', 'O'), ('felicitaties.', 'O')]]
IO: [[('In', 'O'), ('de', 'O'), ('bruisende', 'O'), ('stad', 'O'), ('Amster

<div style="text-align: justify; max-width: 1250px;">
Observant les frases resultants amb els tokens etiquetats per tipus d'entitat per les tres diferents codificacions podem extreure que cap dels models usats per cada tipus d'etiquetatge tenen automatitzada la reconeixença de les últimes paraules, és a dir, les que acaben en punt, com a entitats anomenades tal com passava amb l'idioma espanyol. Totes reconeixen les mateixes entitats, i només s'equivoquen quan reconeixen correctament l'entitat 'Amsterdam, Nederland' perquè cap defineix de manera acertada la classe d'aquesta entitat, 'LOC'. 
</div>

Calculem ara les prediccions per l'idioma neerlandés.

In [34]:
ned_bio = obtenir_entitats_amb_posicions_BIO(predicted_BIO_ned)
palabras_y_etiqueta = [entitats(tupla) for tupla in ned_bio]
print('Etiquetatge BIO:', palabras_y_etiqueta)

ned_io = obtenir_entitats_amb_posicions_IO(predicted_IO_ned)
palabras_y_etiqueta = [entitats(tupla) for tupla in ned_io]
print('Etiquetatge IO:', palabras_y_etiqueta)

ned_bioes = obtenir_entitats_amb_posicions_bioes(predicted_BIOES_ned)
palabras_y_etiqueta = [entitats(tupla) for tupla in ned_bioes]
print('Etiquetatge BIOES:', palabras_y_etiqueta)

Etiquetatge BIO: [('Amsterdam,', 'Nederland,'), ('Tom', 'Dumoulin,'), ('Nederlandse',), ('Dumoulin',), ('Dumoulin',)]
Etiquetatge IO: [('Amsterdam,', 'Nederland,'), ('Tom', 'Dumoulin,'), ('Nederlandse',), ('Dumoulin',), ('Dumoulin',)]
Etiquetatge BIOES: [('Tom', 'Dumoulin,'), ('Amsterdam,', 'Nederland,'), ('Nederlandse',), ('Dumoulin',), ('Dumoulin',)]


In [36]:
print('Etiquetatge BIO:')
resultats(predicted_BIO_ned, text_real_BIO_ned, obtenir_entitats_amb_posicions_BIO)
print('-'*50)

print('\nEtiquetatge IO:')
resultats(predicted_IO_ned, text_real_IO_ned, obtenir_entitats_amb_posicions_IO)
print('-'*50)

print('\nEtiquetatge BIOES:')
resultats(predicted_BIOES_ned, text_real_BIOES_ned, obtenir_entitats_amb_posicions_bioes)
print('-'*50)

Etiquetatge BIO:
Precisió: 0.8
Recall: 0.8
F1-score: 0.8000000000000002
--------------------------------------------------

Etiquetatge IO:
Precisió: 0.8
Recall: 0.8
F1-score: 0.8000000000000002
--------------------------------------------------

Etiquetatge BIOES:
Precisió: 0.8
Recall: 0.8
F1-score: 0.8000000000000002
--------------------------------------------------


Tots presenten els mateixos resultats com ens esperàvem després d'analitzar la reconeixença d'entitats per cada codificació.

## <span style="color:lightblue; font-weight:bold;">Opcional CADEC</span> <a id="opcional_cadec"></a>

In [ ]:
%pip install conllu

In [21]:
class FeatureExtractor_opcional:
    
    """
    Aquesta classe conté el mètode per calcular les features
    que s'utilitzaran per entrenar el model més endavant.
    """
    
    def __init__(self, use_basic_features=False, use_prefix_suffix_features=False, context_features=False, pattern = r'\d+'):
        self.use_basic_features = use_basic_features
        self.use_prefix_suffix_features = use_prefix_suffix_features
        self._pattern = pattern
        self.context_features = context_features
        
    def __str__(self):
        features = []
        if self.use_basic_features:
            features.append("use_basic_features=True")
        if self.use_prefix_suffix_features:
            features.append("use_prefix_suffix_features=True")
        if self.context_features:
            features.append("context_features=True")
        return f"FeatureExtractor({', '.join(features)})"
        
    def _get_features(self, tokens, idx):
        """
        Extract basic features about this word including
            - Current word
            - is it capitalized?
            - Does it have punctuation?
            - Does it have a number?
            - Preffixes up to length 3
            - Suffixes up to length 3
            - paraules prèvies i posteriors amb POS
            - POS-tags

        Note that : we might include feature over previous word, next word etc.

        :return: a list which contains the features
        :rtype: list(str)
        """
            
        token = tokens[idx]
        
        feature_list = []
        
        if self.use_basic_features:
            # Capitalization
            if token[0].isupper():
                feature_list.append("CAPITALIZATION")

            # Number
            if re.search(self._pattern, token) is not None:
                feature_list.append("HAS_NUM")

            # Punctuation
            punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
            if all(unicodedata.category(x) in punc_cat for x in token):
                feature_list.append("PUNCTUATION")

                    
        if self.use_prefix_suffix_features:
            # preffix up to length 3
            if len(token) > 1:
                feature_list.append("PRE_" + token[:1])
            if len(token) > 2:
                feature_list.append("PRE_" + token[:2])
            if len(token) > 3:
                feature_list.append("PRE_" + token[:3])

            # Suffix up to length 3
            if len(token) > 1:
                feature_list.append("SUF_" + token[-1:])
            if len(token) > 2:
                feature_list.append("SUF_" + token[-2:])
            if len(token) > 3:
                feature_list.append("SUF_" + token[-3:])
                
        if self.context_features:
            # Paraules prèvies amb POS
            if idx > 0:
                feature_list.append("anterior1_" + tokens[idx-1])
            if idx > 1:
                feature_list.append("anterior2_" + tokens[idx-2])
                
            # Paraules posteriors amb POS
            if idx < (len(tokens)-1):
                feature_list.append("posterior1_" + tokens[idx+1])
            if idx < (len(tokens)-2):
                feature_list.append("posterior2_" + tokens[idx+2])

            feature_list.append("WORD_" + token)
            
        
        if not self.context_features:
            feature_list.append("WORD_" + token)        
            
        
        return feature_list

In [12]:
def obtenir_token_opcional(fitxer):
    """
    Funció per convertir un text amb el token, POS tag i entitat 
    per cada element en cada frase en un text amb només el token
    per cada element.
    """
    res = []
    for sentence in fitxer:
        frases = []
        for elem1, elem2 in sentence:
            frases.append(elem1)
        res.append(frases)
    return res

In [3]:
#from conllu import parse

# Ruta del archivo .conll de entrenamiento
ruta_entrenamiento_conll = "train.conll"

# Ruta del archivo .conll de prueba
ruta_prueba_conll = "test.conll"

# Cargar datos de entrenamiento
with open(ruta_entrenamiento_conll, "r", encoding="utf-8") as file:
    entrenamiento_conll = file.read()
    opcional_train = entrenamiento_conll

# Cargar datos de prueba
with open(ruta_prueba_conll, "r", encoding="utf-8") as file:
    prueba_conll = file.read()
    opcional_test = prueba_conll


In [6]:
def procesar_datos(fitxer):
    # Dividir los datos por líneas
    lineas = fitxer.strip().split('\n')

    # Inicializar la lista de resultados
    resultados = []

    # Inicializar la lista para cada bloque de datos
    bloque_actual = []

    # Iterar sobre cada línea de los datos
    for linea in lineas:
        # Dividir la línea en sus elementos
        elementos = linea.split('\t')

        # Verificar si la línea está vacía (inicio de un nuevo bloque)
        if len(elementos) == 1:
            # Agregar el bloque actual a los resultados si no está vacío
            if bloque_actual:
                resultados.append(bloque_actual)
                bloque_actual = []  # Reiniciar el bloque actual

        # Si la línea no está vacía, procesarla
        else:
            # Obtener el tipo de entidad según la columna
            tipo_entidad = None
            for i in range(1, len(elementos)):
                if elementos[i] != 'O':
                    tipo_entidad = 'ADR' if i == 1 else ('Di' if i == 2 else ('Dr' if i == 3 else ('S' if i == 4 else 'F')))
                    indx = i
                    break

            # Construir la etiqueta de entidad
            if tipo_entidad:
                if elementos[indx].startswith(('B')):
                    entidad = 'B'
                elif elementos[indx].startswith(('I')):
                    entidad = 'I'

                bloque_actual.append((elementos[0], entidad + '-' + tipo_entidad))
            else:
                bloque_actual.append((elementos[0], 'O'))

    # Agregar el último bloque a los resultados si no está vacío
    if bloque_actual:
        resultados.append(bloque_actual)

    return resultados

In [7]:
opcional_train = procesar_datos(opcional_train)
opcional_test = procesar_datos(opcional_test)

In [14]:
""" 
Diria que aixo no fa falta, perquè en el opcional_train no 
tenim els POS_tags, només les entitats i les seves codificacions. 
"""

model_tagger_POS_eng = CRFTagger()

# Entrenem el model per predir els POS que corresponen a cada token    
model_tagger_POS_eng.train(opcional_train, 'model_POS_eng.crf.tagger')    


testa_eng_pre_tag = obtenir_token_opcional(opcional_test)
    
predicted = model_tagger_POS_eng.tag_sents(testa_eng_pre_tag)

predictions = [elem[1] for sentence in predicted for elem in sentence]
real_label = [elem[1] for sentence in opcional_test for elem in sentence]

print(accuracy_score(predictions, real_label))

0.8954845923636005


In [22]:
param_combinations_eng = [
    None,
    FeatureExtractor_opcional(),
    FeatureExtractor_opcional(True),
    FeatureExtractor_opcional(True, True),
    FeatureExtractor_opcional(True, True, True)
]

train_eng_BIO = opcional_train
test_eng_real = opcional_test

test_eng_pre_tag = obtenir_token_opcional(opcional_test)


def model_entrenament(train_eng_BIO_tag, extractor, test_eng_pre_tag):
    if extractor == None:
        model_BIO = CRFTagger()
        model_BIO.train(train_eng_BIO_tag, 'model_opcional.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(test_eng_pre_tag)
    
    else:
        model_BIO = CRFTagger(feature_func=extractor._get_features)
        model_BIO.train(train_eng_BIO_tag, 'model_opcional.crf.tagger')
        
        predicted_BIO = model_BIO.tag_sents(test_eng_pre_tag)
    
    return resultats(predicted_BIO, test_eng_real, obtenir_entitats_amb_posicions_BIO)

for param in param_combinations_eng:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_eng_BIO, param, test_eng_pre_tag)
    print('-'*50)
    

Model: Features per defecte de CRFTagger
Precisió: 0.6053067993366501
Recall: 0.3967391304347826
F1-score: 0.47931713722915303
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.608324439701174
Recall: 0.30978260869565216
F1-score: 0.4105149441843716
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6102403343782654
Recall: 0.3173913043478261
F1-score: 0.4175902752949589
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6141141141141141
Recall: 0.44456521739130433
F1-score: 0.5157629255989912
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, context_features=True)
Precisió: 0.6603508771929825
Recall: 0.5114130434782609
F1-score: 0.5764165390505359
--------------------------------------------------


In [23]:
def convert_to_io_opcional(train_data_bio):
    """
    Funció per convertir les dades del format BIO al format IO.
    Argument:
    train_data_bio: una llista de frases, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIO).
    Retorna:
    Una llista de frases en format IO, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta IO).
    """
    train_data_io = []
    for sentence in train_data_bio:
        io_tags = []
        for word, bio_tag in sentence:
            if bio_tag == 'O':
                io_tags.append('O')
            elif bio_tag.startswith('B-'):
                io_tags.append('I' + bio_tag[1:])
            else:
                io_tags.append(bio_tag)
        
        train_data_io.append([(word, io_tag) for (word, bio_tag), io_tag in zip(sentence, io_tags)])
    return train_data_io

In [26]:

train_eng_IO = convert_to_io_opcional(opcional_train)
test_eng_real = convert_to_io_opcional(opcional_test)

test_eng_pre_tag = obtenir_token_opcional(opcional_test)


def model_entrenament(train_eng_IO_tag, extractor, test_eng_pre_tag):
    if extractor == None:
        model_IO = CRFTagger()
        model_IO.train(train_eng_IO_tag, 'model_opcional.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(test_eng_pre_tag)
    
    else:
        model_IO = CRFTagger(feature_func=extractor._get_features)
        model_IO.train(train_eng_IO_tag, 'model_opcional.crf.tagger')
        
        predicted_IO = model_IO.tag_sents(test_eng_pre_tag)
    
    return resultats(predicted_IO, test_eng_real, obtenir_entitats_amb_posicions_IO)

for param in param_combinations_eng:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_eng_IO, param, test_eng_pre_tag)
    print('-'*50)
    

Model: Features per defecte de CRFTagger
Precisió: 0.5913621262458472
Recall: 0.37160751565762007
F1-score: 0.45641025641025645
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.5845665961945031
Recall: 0.2886221294363257
F1-score: 0.3864430468204053
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.5886524822695035
Recall: 0.30323590814196244
F1-score: 0.40027557698932137
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6059925093632959
Recall: 0.4222338204592902
F1-score: 0.4976930175330668
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, context_features=True)
Precisió: 0.6535057862491491
Recall: 0.5010438413361169
F1-score: 0.5672082717872969
--------------------------------------------------


In [27]:
def convert_to_bioes_opcional(train_data_bio):
    """
    Funció per convertir dades de format BIO a format BIOES.
    
    Argument:
    train_data_bio: una llista de frases, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIO).
    
    Retorna:
    Una llista de frases en format BIOES, on cada frase és una llista de tuples (paraula, etiqueta POS, etiqueta BIOES).
    """
    train_data_bioes = []
    for sentence in train_data_bio:
        bioes_tags = []
        for i, (word, bio_tag) in enumerate(sentence):
            if bio_tag == 'O':
                bioes_tags.append('O')
            elif bio_tag.startswith('B-'):
                if i == len(sentence) - 1 or sentence[i + 1][1] != 'I' + bio_tag[1:]:
                    bioes_tags.append('S' + bio_tag[1:])  # Single
                else:
                    bioes_tags.append('B' + bio_tag[1:])  # Begin
            elif bio_tag.startswith('I-'):
                if i == len(sentence) - 1 or sentence[i + 1][1] != 'I' + bio_tag[1:]:
                    bioes_tags.append('E' + bio_tag[1:])  # End
                else:
                    bioes_tags.append('I' + bio_tag[1:])  # Inside
            else:
                raise ValueError("Etiqueta BIO incorrecta: {}".format(bio_tag))
        
        train_data_bioes.append([(word, bioes_tag) for (word, bio_tag), bioes_tag in zip(sentence, bioes_tags)])
    return train_data_bioes

In [30]:
train_eng_BIOES = convert_to_bioes_opcional(opcional_train)
test_eng_real = convert_to_bioes_opcional(opcional_test)

test_eng_pre_tag = obtenir_token_opcional(opcional_test)


def model_entrenament(train_eng_BIOES_tag, extractor, test_eng_pre_tag):
    if extractor == None:
        model_BIOES = CRFTagger()
        model_BIOES.train(train_eng_BIOES_tag, 'model_opcional.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(test_eng_pre_tag)
    
    else:
        model_BIOES = CRFTagger(feature_func=extractor._get_features)
        model_BIOES.train(train_eng_BIOES_tag, 'model_opcional.crf.tagger')
        
        predicted_BIOES = model_BIOES.tag_sents(test_eng_pre_tag)
    
    return resultats(predicted_BIOES, test_eng_real, obtenir_entitats_amb_posicions_bioes)

for param in param_combinations_eng:
    print(f'Model: {str(param)}') if param != None else print(f'Model: Features per defecte de CRFTagger')
    model_entrenament(train_eng_BIOES, param, test_eng_pre_tag)
    print('-'*50)
    

Model: Features per defecte de CRFTagger
Precisió: 0.6179245283018868
Recall: 0.40494590417310666
F1-score: 0.48926237161531283
--------------------------------------------------
Model: FeatureExtractor()
Precisió: 0.63579604578564
Recall: 0.3147861926841834
F1-score: 0.42108890420399725
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True)
Precisió: 0.6367305751765893
Recall: 0.3250901597114889
F1-score: 0.4304229195088677
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True)
Precisió: 0.6296028880866426
Recall: 0.44925296239052037
F1-score: 0.524353577871317
--------------------------------------------------
Model: FeatureExtractor(use_basic_features=True, use_prefix_suffix_features=True, context_features=True)
Precisió: 0.6646300067888663
Recall: 0.5043791859866048
F1-score: 0.5735207967193907
--------------------------------------------------
